<a href="https://colab.research.google.com/github/robertalanm/TT_tests/blob/master/new_actions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Requisites

> ## Old Tensortrade

In [3]:
!pip install git+https://github.com/robertalanm/Tensortrade-old

  Cloning https://github.com/robertalanm/Tensortrade-old to /tmp/pip-req-build-hherwi4m
  Running command git clone -q https://github.com/robertalanm/Tensortrade-old /tmp/pip-req-build-hherwi4m
  Created wheel for tensortrade: filename=tensortrade-0.0.1a17-cp36-none-any.whl size=54534 sha256=4efaea3dec894da3c54a12ba2cec54dd6fcaf587725d6131bc80039f6673b03a
  Stored in directory: /tmp/pip-ephem-wheel-cache-1sp35rwr/wheels/92/a4/ec/dbcdbc6e8596c3d995ec5009d5bbc60fdf1560ab25dcca351a
Successfully built tensortrade


In [0]:
%cd /content
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzf ta-lib-0.4.0-src.tar.gz
%cd ta-lib/
!./configure --prefix=/usr
!make
!sudo make install
%cd /content

In [40]:
!pip install ccxt stochastic tensorforce==0.5.2
!pip install git+https://github.com/mrjbq7/ta-lib

     |████████████████████████████████| 317kB 5.1MB/s 
  Found existing installation: tensorforce 0.4.4
    Uninstalling tensorforce-0.4.4:
      Successfully uninstalled tensorforce-0.4.4


  Cloning https://github.com/mrjbq7/ta-lib to /tmp/pip-req-build-4fo6rsm7
  Running command git clone -q https://github.com/mrjbq7/ta-lib /tmp/pip-req-build-4fo6rsm7
  Created wheel for TA-Lib: filename=TA_Lib-0.4.18-cp36-cp36m-linux_x86_64.whl size=1528881 sha256=1e3a74fe829c408d0a5ca9d6290f4c7a1f42c79184a1e8edce7eaaf515e7eb4b
  Stored in directory: /tmp/pip-ephem-wheel-cache-lw3gbogl/wheels/7c/09/16/87b01ae949260f15a1821ae29d58a15642c203fcc03d049fcb
Successfully built TA-Lib


> ## Latest Tensortrade

In [7]:
#!pip install git+https://github.com/robertalanm/TT_tests
!pip install git+https://github.com/notadamking/tensortrade

  Cloning https://github.com/notadamking/tensortrade to /tmp/pip-req-build-836lq9n9
  Running command git clone -q https://github.com/notadamking/tensortrade /tmp/pip-req-build-836lq9n9
     |████████████████████████████████| 266kB 5.0MB/s 
     |████████████████████████████████| 5.3MB 10.9MB/s 
  Created wheel for tensortrade: filename=tensortrade-0.1.0rc1-cp36-none-any.whl size=92945 sha256=f087deff3dfddfeb63ef006483630c261dd61950f8af2e85bf070918bddc5ed3
  Stored in directory: /tmp/pip-ephem-wheel-cache-zw0_gkf_/wheels/7c/ed/66/c708e8f58e006fa34ffc022ac50de732e7a2aabb91654774f5
  Created wheel for pyyaml: filename=PyYAML-5.1.2-cp36-cp36m-linux_x86_64.whl size=44104 sha256=90fde1115e28f20c2394c87960a590a70bc432d8c4a78d7c0264de0ac276b90d
  Stored in directory: /root/.cache/pip/wheels/d9/45/dd/65f0b38450c47cf7e5312883deb97d065e030c5cca0a365030
Successfully built tensortrade pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyY

In [1]:
!pip install ccxt stochastic tensorforce==0.5.2 ta

# Data Fetcher

> ## pull data

In [0]:
import requests
import time
import csv
import json
import sys, os
import datetime

#fetching data from bitstamp and write to a csv file
F_Z = "%Y-%m-%dT%H:%M:%S.%fZ"
'''
currency_pairs = ['BTC-USD', 'BTC-EUR', 'BTC-GBP',
                  'ETH-USD', 'ETH-EUR', 'ETH-GBP',
                  'BCH-USD', 'BCH-EUR', 'BCH-GBP',
                  'LTC-USD', 'LTC-EUR', 'LTC-GBP',
                    'ETC-USD', 'ETC-EUR', 'ETC-GBP',

                  ]
'''

currency_pairs = ['BTC-USD']

OB_url = 'https://api.pro.coinbase.com/products/%s/book?level=2'
trans_url = 'https://api.pro.coinbase.com/products/%s/trades'

trimmed_ob_limit = 30
time_sleep = 25
last_transaction = {}
for i in currency_pairs:
    last_transaction[i] = {'trade_id': '', 'time': str(datetime.datetime.utcnow().strftime(F_Z))}

def try_parsing_date(text):
    for fmt in ('%Y-%m-%dT%H:%M:%S.%fZ', '%Y-%m-%dT%H:%M:%SZ'):
        try:
            return datetime.datetime.strptime(text, fmt)
        except ValueError:
            pass
    raise ValueError('no valid date format found')


while True:
    for curr_pair in currency_pairs:
        try:
            resp = requests.get(url=OB_url % curr_pair)
            ob_snapshot = resp.json()
            transactions = requests.get(url=trans_url % curr_pair).json()
            transaction_recent = transactions[0] if len(transactions) > 0 else {'trade_id': ''}

            last_transaction_time = try_parsing_date(last_transaction[curr_pair]['time']).timestamp()

            filtered_list = [
                {'side': i['side'], 'trade_id': i['trade_id'], 'price': i['price'],
                 'size': i['size'], 'time': try_parsing_date(i['time']).timestamp()
                 }
                for i in transactions if \
                                try_parsing_date(i['time']).timestamp() >
                             last_transaction_time]

            recorded_transaction = json.dumps(filtered_list)  \
                if last_transaction[curr_pair]['trade_id'] != transaction_recent['trade_id'] else ''

            filename = 'cb_%s.csv' % curr_pair

            with open(filename, 'a') as csvfile:
                fieldnames = ['timestamp', 'transactions', 'ob']
                timestamp = int(time.time())
                trimmed_ob = json.dumps({'bid': [[i[0], i[1]] for i in ob_snapshot['bids'][:trimmed_ob_limit]],
                              'ask': [[i[0], i[1]] for i in ob_snapshot['asks'][:trimmed_ob_limit]]})
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writerow({'timestamp': timestamp, 'transactions': recorded_transaction, 'ob': trimmed_ob})
                last_transaction[curr_pair] = transaction_recent
        except Exception as e:
            exc_type, exc_obj, exc_tb = sys.exc_info()
            fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
            print(exc_type, fname, exc_tb.tb_lineno, e)
        time.sleep(0.5)
    time.sleep(time_sleep)

KeyboardInterrupt: ignored

> ## Feature Transformation

In [3]:

import pandas as pd
import numpy as np
import json
import ast
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.style.use('seaborn')

# df = pd.read_csv('bitstamp_btcusd.csv', header=None, names=['timestamp', 'trades', 'book'])
df = pd.read_csv('cb_BTC-USD.csv', header=None, names=['timestamp', 'trades', 'book'])

df['trades'] = df['trades'].apply(lambda d: json.loads(d) if isinstance(d, str) else [])
df['book'] = df['book'].apply(lambda d: json.loads(d))
df.iloc[1]
df.shape
# df.iloc[1382]['book']

(4253, 3)

In [0]:
import math
def trades_to_weighted_px(trades_arr):
    notional = 0
    qty = 0
    for trade in trades_arr:
        notional += float(trade['price']) * float(trade['size'])
        qty += float(trade['size'])
    
    return notional / (qty + 1e-16)

def trades_to_latest_px(trades_arr):
    max_time = 0
    trade_px = 0
    for trade in trades_arr:
        timestamp = int(trade['time'])
        if timestamp > max_time:
            trade_px = float(trade['price'])
    return trade_px

def trades_to_vol(trades):
    qty = sum([float(trade['size']) for trade in trades])
    return qty 

def trades_to_high(row):
    if len(row['trades']) > 0:
        return max(float(trade['price']) for trade in row['trades'])
    else:
        return row['latest_trade_px']

# def trades_to_low(trades):
#     return min(float(trade['price']) for trade in trades)

def trades_to_low(row):
    if len(row['trades']) > 0:
        return min(float(trade['price']) for trade in row['trades'])
    else:
        return row['latest_trade_px']

def interpolate_tradepx(df):
    for i in range(len(df)):
        trade_px = df.iloc[i]['trade_px']
        if trade_px == 0:
            if i == 0:
                new_trade_px = 0
                for j in range(len(df)):
                    if df.iloc[j]['trade_px'] != 0:
                        new_trade_px = df.iloc[j]['trade_px']
                        break
                df.iloc[i]['trade_px'] = new_trade_px
            else:
                df.iloc[i]['trade_px'] = df.iloc[i - 1]['trade_px'] 


def get_log_close(df):
  for i in range(len(df)):
    if i == 0:
      return 0
    if i > 0:
      df['log_close'][i] = (math.log(df['bid_px'][i]) - math.log(df['ask_px'][i].shift(1)))



def get_base_features(old_df):
    df = old_df.copy()
    
    #generate base data
    df['trade_px'] = df['trades'].apply(trades_to_weighted_px)
    df['volume'] = df['trades'].apply(trades_to_vol)
    df['latest_trade_px'] = df['trades'].apply(trades_to_latest_px)
    df['latest_trade_px'] = df['latest_trade_px'].replace(to_replace=0, method='ffill')
    df['bid_px'] = df['book'].apply(lambda x: float(x['bid'][0][0]))
    df['ask_px'] = df['book'].apply(lambda x: float(x['ask'][0][0]))
    df['bid_vol'] = df['book'].apply(lambda x: float(x['bid'][0][1]))
    df['ask_vol'] = df['book'].apply(lambda x: float(x['ask'][0][1]))
    df['mid_px'] = (df['bid_px'] + df['ask_px'])/2

    #TA modules
    df['High'] = df.apply(trades_to_high, axis=1)
    df['Low'] = df.apply(trades_to_low, axis=1)
    df['Timestamp'] = df['timestamp']
    df['Close'] = df['latest_trade_px']
    df['Open'] = df['latest_trade_px'].shift(1)
    df['Volume'] = df['volume']
    
    # y
    df['change_pct_latest_px'] = df['latest_trade_px'].pct_change()
    df['px_change'] = df['latest_trade_px'].diff()
    
    # generate features
    df['spread'] = df['ask_px'] - df['bid_px']
    df['l1imbalance'] = (df['bid_vol'] - df['ask_vol']) / ((df['bid_vol'] + df['ask_vol']) + 1e-9)
    df['bid_delta'] = df['bid_px'] - df['bid_px'].shift(1)
    df['ask_delta'] = df['ask_px'] - df['ask_px'].shift(1)
    df['bid_vol_delta'] = df['bid_vol'] - df['bid_vol'].shift(1)
    df['ask_vol_delta'] = df['ask_vol'] - df['ask_vol'].shift(1)
    df['bid_vol_acceleration'] = ((df['bid_vol_delta'] - df['bid_vol_delta'].shift(1)) / 
            (df['bid_vol_delta'].shift(1) + 1e-9))
    df['ask_vol_acceleration'] = ((df['ask_vol_delta'] - df['ask_vol_delta'].shift(1)) / 
            (df['ask_vol_delta'].shift(1) + 1e-9))
    
    df['time_delta'] = df['timestamp'] - df['timestamp'].shift(1)
    df['time_delta'] = df['time_delta'].apply(lambda seconds: np.nan if seconds > 40 else seconds)
    
    #l2 features
    
    
    df = df.replace([np.inf, -np.inf], np.nan)
    
    return df

In [0]:
new_df = get_base_features(df).dropna()
new_df.drop(df.iloc[:, 1:3], inplace = True, axis = 1)
new_df.shape
new_df
# new_df.rolling(window=5).mean()
new_df.to_csv('NEW_DF.csv')

# Running Code

> ## Models

> ## Impl

> > ### Use with Tensorforce 0.5.2

In [50]:
import tensorflow as tf
import pandas as pd
import ccxt

from stable_baselines.common.policies import MlpLnLstmPolicy
from stable_baselines import PPO2, ACER

from tensorforce.agents import Agent

from tensortrade.strategies import StableBaselinesTradingStrategy, TensorforceTradingStrategy
from tensortrade.environments import TradingEnvironment
from tensortrade.rewards import RiskAdjustedReturns
from tensortrade.actions import ManagedRiskOrders, DynamicOrders
from tensortrade.instruments import Quantity, TradingPair, BTC, USD
from tensortrade.orders.criteria import Stop, StopDirection
from tensortrade.wallets import Wallet, Portfolio
from tensortrade.exchanges.simulated import SimulatedExchange
from tensortrade.exchanges.live import CCXTExchange
from tensortrade.features.stationarity import LogDifference
from tensortrade.features.scalers import MinMaxNormalizer
from tensortrade.features import FeaturePipeline

WINDOW_SIZE = 4
#PRICE_COLUMN = 'change_pct_latest_px'
PRICE_COLUMN = 'close'

normalize = MinMaxNormalizer(inplace=True)
difference = LogDifference(inplace=True)
feature_pipeline = FeaturePipeline(steps=[normalize, difference])

action_scheme = ManagedRiskOrders(pairs=[USD/BTC], stop_loss_percentages=0.05, take_profit_percentages=0.6)
reward_scheme = RiskAdjustedReturns(return_algorithm="sharpe")

csv_file = tf.keras.utils.get_file('Coinbase_BTCUSD_1h.csv', 'http://www.cryptodatadownload.com/cdd/Coinbase_BTCUSD_1h.csv')
#ohlcv_data = pd.read_csv('NEW_DF.csv', skiprows=0, index_col="timestamp")
ohlcv_data = pd.read_csv(csv_file, skiprows=1, index_col="Date")
ohlcv_data.columns = map(str.lower, ohlcv_data.columns)
ohlcv_data = ohlcv_data.rename(columns={'volume btc': 'volume'})


#coinbase = ccxt.coinbasepro()
#exchange = CCXTExchange(exchange=coinbase, base_instrument='USD')
exchange = SimulatedExchange(data_frame=ohlcv_data, price_column=PRICE_COLUMN, randomize_time_slices=True)

wallets = [(exchange, USD, 100), (exchange, BTC, 0)]

portfolio = Portfolio(base_instrument=USD, wallets=wallets)

environment = TradingEnvironment(exchange=exchange,
                                 portfolio=portfolio,
                                 action_scheme=action_scheme,
                                 reward_scheme=reward_scheme,
                                 feature_pipeline=feature_pipeline,
                                 window_size=WINDOW_SIZE,
                                 observe_wallets=[USD, BTC])

print('Observation Data:')
print(environment.observation_columns)

model = PPO2
#model = ACER
policy = MlpLnLstmPolicy
params = { "learning_rate": 1e-5, 'nminibatches': 1,  }
#params = { "learning_rate": 1e-5,  }

base_strategy = StableBaselinesTradingStrategy(environment=environment,
                                          model=model,
                                          policy=policy,
                                          model_kwargs=params)


agent = Agent.create(
    agent='dueling_dqn',
    states=dict(type='float', shape=(4,10)),
    actions=dict(type='int', num_values=3),
    max_episode_timesteps=100,
    #memory=10000,
    #update=dict(unit='timesteps', batch_size=64),
    #optimizer=dict(type='adam', learning_rate=3e-4),
    #policy=dict(network='auto'),
    #objective='policy_gradient',
    #reward_estimation=dict(horizon=20)
)

force_strategy = TensorforceTradingStrategy(environment=environment, 
                                      agent=agent, max_episode_timesteps=20000)

force_strategy.run(steps=1000)
print(f"Tensorforce net worth: {portfolio.net_worth}")
base_strategy.run(steps=1000)
print(f"Baselines net worth: {portfolio.net_worth}")


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Observation Data:
['open' 'high' 'low' 'close' 'volume' 'volume usd' 'USD' 'BTC'
 'USD_pending' 'BTC_pending']








Timesteps:   0%|          | 0/1000 [00:00<?, ?it/s, mean_reward=n/a]/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s








Finished running strategy.
Total episodes: 0 (1000 timesteps).
Average reward: -19.26090056459747.
Tensorforce net worth: 102.60238106685298
Baselines net worth: 106.2635740482776


> > ### Use with Tensorforce 0.4.4

In [6]:
import os
import sys
import warnings
import numpy
import pandas as pd

def warn(*args, **kwargs):
    pass

warnings.warn = warn
warnings.simplefilter(action='ignore', category=FutureWarning)
numpy.seterr(divide = 'ignore') 
#%% Module version check
import tensorforce
import tensorflow as tf

#%% Actions & Rewards
from tensortrade.strategies import TensorforceTradingStrategy
#from tensortrade.rewards import PositionReward
#from tensortrade.actions import FuturePositionStrategy

from tensortrade.actions import DiscreteActionStrategy

action_scheme = DiscreteActionStrategy(n_actions=20, instrument_symbol='BTC')

from tensortrade.rewards import RiskAdjustedReturnStrategy

reward_scheme = RiskAdjustedReturnStrategy(return_algorithm="sortino")
#reward_strategy = PositionReward()
#action_strategy = FuturePositionStrategy(n_actions = 11)

#%% Feature Pipeline
from tensortrade.features.stationarity import FractionalDifference
from tensortrade.features.scalers import MinMaxNormalizer
from tensortrade.features import FeaturePipeline

normalize = MinMaxNormalizer(inplace=True)
difference = FractionalDifference(difference_order=0.6,
                                  inplace=True)
feature_pipeline = FeaturePipeline(steps=[])

#%% Data Input (Exchanges)
from tensortrade.exchanges.simulated import SimulatedExchange
#from tensortrade.exchanges.simulated.future_exchange_position import FutureExchangePosition
#csv_file = tf.keras.utils.get_file(
#    'Coinbase_BTCUSD_1h.csv', 'http://www.cryptodatadownload.com/cdd/Coinbase_BTCUSD_1h.csv')
#data = pd.read_csv(csv_file, skiprows=1, index_col="Date")
data = pd.read_csv('NEW_DF.csv',index_col = 0)
data.columns = map(str.lower, data.columns)

#data = data[data.index % 60 == 0]
#data = data.reset_index(drop = True)


#exchange = FutureExchangePosition(data, base_instrument = 'USD', exclude_close = True,
#                          initial_balance = 50000, observe_position = True)

exchange = SimulatedExchange(base_instrument='USD',
                             data_frame=data,
                             price_column='close',
                             window_size=1,
                             pretransform=False)

#%% Agent Specification
network_spec = [
    dict(type='dense', size=128, activation="tanh"),
    dict(type='dense', size=64, activation="tanh"),
    dict(type='dense', size=32, activation="tanh")
]

agent_spec = {
    "type": "a2c",
    "learning_rate": 0.0003,
    "discount": 1.0,
    "estimate_terminal": False,
    "max_episode_timesteps": 20000,
    "network": network_spec,
    "batch_size": 100,
    "update_frequency":100
}


#%%Environment Setup
from tensortrade.environments import TradingEnvironment

environment = TradingEnvironment(exchange=exchange,
                                 action_strategy=action_scheme,
                                 reward_strategy=reward_scheme,
                                 feature_pipeline=feature_pipeline)


strategy = TensorforceTradingStrategy(environment=environment, 
                                      agent_spec=agent_spec, save_best_agent=False)
#%%Start Over
performance = strategy.run(episodes=100, evaluation=False)

#manually store agent
#strategy.save_agent(directory = 'save/', filename = '01')

#%% Restore and Continue 
'''
strategy.restore_agent(directory = 'save/', filename = '01')
performance = strategy.run(episodes=(strategy._runner.agent.episodes + 20), evaluation=False)
'''
#%% Analysis
performance =(pd.DataFrame(data = self._performance.T, columns = ['balance','net_worth','open_amount','price']))
learning_curve = pd.Series(strategy._runner.episode_rewards)
learning_curve.plot()

AttributeError: ignored